In [2]:
import os

# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4
! pip install --ignore-installed -q spark-nlp==2.5.3

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
     |████████████████████████████████| 215.7MB 22kB/s 
     |████████████████████████████████| 204kB 55.9MB/s 
     |████████████████████████████████| 133kB 3.3MB/s 


In [3]:
# Run this in console to keep Co Lab Running
'''
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
'''

'\nfunction ConnectButton(){\n    console.log("Connect pushed"); \n    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() \n}\nsetInterval(ConnectButton,60000);\n'

In [4]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from pyspark.ml import Pipeline
from pyspark.ml.tuning import TrainValidationSplit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Start Spark session
spark = sparknlp.start()

In [6]:
# Best practice to print versions
print("Spark NLP version", sparknlp.version())
print("Apache Spark version", spark.version)

Spark NLP version 2.5.3
Apache Spark version 2.4.4


In [7]:
# Yin genius way of loading Amazon Data
file = 'amazon_reviews_us_Pet_Products_v1_00.tsv.gz'
os.system(f'wget https://s3.amazonaws.com/amazon-reviews-pds/tsv/{file}')
df = pd.read_csv(file, compression='gzip', header=0, sep='\t', error_bad_lines=False)

b'Skipping line 21404: expected 15 fields, saw 22\nSkipping line 31839: expected 15 fields, saw 22\nSkipping line 37779: expected 15 fields, saw 22\nSkipping line 38344: expected 15 fields, saw 22\nSkipping line 61449: expected 15 fields, saw 22\n'
b'Skipping line 80975: expected 15 fields, saw 22\nSkipping line 85603: expected 15 fields, saw 22\nSkipping line 95950: expected 15 fields, saw 22\nSkipping line 105132: expected 15 fields, saw 22\nSkipping line 115378: expected 15 fields, saw 22\n'
b'Skipping line 152632: expected 15 fields, saw 22\nSkipping line 159310: expected 15 fields, saw 22\nSkipping line 162724: expected 15 fields, saw 22\nSkipping line 168588: expected 15 fields, saw 22\nSkipping line 170412: expected 15 fields, saw 22\nSkipping line 187169: expected 15 fields, saw 22\n'
b'Skipping line 205461: expected 15 fields, saw 22\nSkipping line 210928: expected 15 fields, saw 22\nSkipping line 213691: expected 15 fields, saw 22\nSkipping line 228697: expected 15 fields, sa

In [8]:
df1= df
#headline_updated = df1["review_headline"].str.contains("One Star|Two Stars|Three Stars|Four Stars|Five Stars",na=False)
#df1.drop(headline_updated[headline_updated].index,axis=0,inplace=True)
df1[["star_rating","review_body"]]= df1[["star_rating","review_body"]].astype(str)
df1.star_rating.value_counts()


5.0    1643151
4.0     380780
1.0     248519
3.0     216335
2.0     151067
nan          1
Name: star_rating, dtype: int64

In [9]:
df1.loc[df1["star_rating"]=="2.0","star_rating"]="3.0" 
df1.loc[df1["star_rating"]=="4.0","star_rating"]="3.0"
df_1 = df1.loc[df1["star_rating"]=="1.0"]
#df_2 = df1.loc[df1["star_rating"]=="2.0"]
df_3 = df1.loc[df1["star_rating"]=="3.0"]
#df_4 = df1.loc[df1["star_rating"]=="4.0"]
df_5 = df1.loc[df1["star_rating"]=="5.0"]

df_star1= df_1.sample(n = 151067, replace = True) 
#df_star2= df_2.sample(n = 151067, replace = True) 
df_star3= df_3.sample(n = 151067, replace = True) 
#df_star4= df_4.sample(n = 151067, replace = True) 
df_star5= df_5.sample(n = 151067, replace = True) 

frames = [df_star1, df_star3,df_star5]

df_sample = pd.concat(frames)
df_sample[["star_rating","review_body"]]= df[["star_rating","review_body"]].astype(str)



In [10]:
df_sample["review_body"]= df_sample["review_body"].str.lower()
#set(reviews([1])

for i in range(10):
  #print(df_sample.iloc[i,13])
  df_sample.iloc[i,13] = ' '.join(list(set(df_sample.iloc[i,13].split())))
  #reviews.at[i,'review_body']=(" ".join(list(set(reviews.at[i,'review_body']))))
df_sample['review_body'].head(10)

2497549    again. few than won't it and less leak. produc...
2038876    hours effort. and less off. product worth the ...
2412215    crate the or now. with something them used new...
2375521    split way because the fell supposed or pet spa...
1463497    total try changed being and perforations, the ...
2200999    />we literally thought work.<br and would the ...
646961     fence fine and got killed the fell until worke...
1480460    safe way and because not. disappointed i made ...
2320       fence reviewes wrote far another the who work ...
468473     please into and the product i contact from hav...
Name: review_body, dtype: object

In [11]:
df_sample["length_comment"]= df["review_body"].str.split().str.len()

print(df_sample["length_comment"])

2497549     19
2038876     28
2412215    143
2375521    131
1463497     97
          ... 
506277      31
907387     196
2310499    366
1229393      7
1843681     79
Name: length_comment, Length: 453201, dtype: int64


In [13]:
# Select out features and labels
entireDataset = spark.createDataFrame(df_sample[['star_rating', 'review_body']])

In [14]:
# Check data types
entireDataset.dtypes

[('star_rating', 'string'), ('review_body', 'string')]

In [15]:
# View col names
entireDataset.columns

['star_rating', 'review_body']

In [16]:
# Change object to string
entireDataset.review_body = entireDataset.review_body.astype('string')

In [17]:
# Prepare for NLP pipeline with initial DocumentAssembler
# Content is inside review_body
document = DocumentAssembler() \
  .setInputCol("review_body") \
  .setOutputCol("document")

In [18]:
# Using sentence detector (pretrained)
# Sentence detector
use = UniversalSentenceEncoder.pretrained() \
  .setInputCols(["document"]) \
  . setOutputCol("sentence_embeddings")

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [19]:
# Our labels / classes are the 1 - 5 star ratings
# These are in the star_rating col

classifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("star_rating")\
  .setMaxEpochs(30)\
  .setEnableOutputLogs(True)

In [20]:
# Declare the pipeline
use_clf_pipeline = Pipeline(
  stages = [
    document,
    use,
    classifierdl
  ]
)



In [21]:
# Split into training and testing data
trainDataset, testDataset = entireDataset.randomSplit([0.75, 0.25], seed=12345)


In [22]:
# Begin fitting on the data
use_pipelineModel = use_clf_pipeline.fit(trainDataset)

In [23]:
# Set predictions on testDataset
predictions = use_pipelineModel.transform(testDataset)
predictions.count()
predictions.show()

+-----------+--------------------+--------------------+--------------------+--------------------+
|star_rating|         review_body|            document| sentence_embeddings|               class|
+-----------+--------------------+--------------------+--------------------+--------------------+
|        1.0|!my order came in...|[[document, 0, 46...|[[sentence_embedd...|[[category, 0, 46...|
|        1.0|#1 = does the pic...|[[document, 0, 55...|[[sentence_embedd...|[[category, 0, 55...|
|        1.0|#1 = does the pic...|[[document, 0, 55...|[[sentence_embedd...|[[category, 0, 55...|
|        1.0|$13.99, and it la...|[[document, 0, 10...|[[sentence_embedd...|[[category, 0, 10...|
|        1.0|$13.99, and it la...|[[document, 0, 10...|[[sentence_embedd...|[[category, 0, 10...|
|        1.0|$14 bucks for thi...|[[document, 0, 44...|[[sentence_embedd...|[[category, 0, 44...|
|        1.0|$25.00 for a 5 in...|[[document, 0, 12...|[[sentence_embedd...|[[category, 0, 12...|
|        1.0|$290 as

In [24]:
# Begin fitting on the data
use_pipelineModel = use_clf_pipeline.fit(trainDataset)

In [25]:
# Set predictions on testDataset
predictions = use_pipelineModel.transform(testDataset)

In [26]:

# Run predictions
predictions.select('star_rating','review_body', 'class.result').show(10, truncate=80)

+-----------+--------------------------------------------------------------------------------+------+
|star_rating|                                                                     review_body|result|
+-----------+--------------------------------------------------------------------------------+------+
|        1.0|!my order came in two packages.  the first pack was ok, but the second pack d...| [1.0]|
|        1.0|#1 = does the picture and the description match?  this is not the description...| [1.0]|
|        1.0|#1 = does the picture and the description match?  this is not the description...| [1.0]|
|        1.0|$13.99, and it lasted less than 5 minutes, before my dog chewed it up, gettin...| [1.0]|
|        1.0|$13.99, and it lasted less than 5 minutes, before my dog chewed it up, gettin...| [1.0]|
|        1.0|                                   $14 bucks for this turd? worth 4-5 bucks max.| [1.0]|
|        1.0|$25.00 for a 5 inch plastic cup?? are you people insane? what makes t

In [27]:
# Import reports for quantifying results
from sklearn.metrics import classification_report, accuracy_score

df = use_pipelineModel.transform(testDataset).select('star_rating','review_body', 'class.result').toPandas()

df['result'] = df['result'].apply(lambda x: x[0])

In [28]:
df.result = df.result.astype(str)

In [29]:
print(classification_report(df['star_rating'], df.result))
print(accuracy_score(df['star_rating'], df.result))

              precision    recall  f1-score   support

         1.0       0.77      0.78      0.77     37909
         3.0       0.60      0.60      0.60     37667
         5.0       0.75      0.74      0.74     37560

    accuracy                           0.71    113136
   macro avg       0.71      0.71      0.71    113136
weighted avg       0.71      0.71      0.71    113136

0.7056639796351294
